In [ ]:
import os
from pydantic import BaseModel
import asyncio
from agents import Agent, Runner, function_tool, trace
import smtplib
from email.message import EmailMessage

# 🔑 API keys (read from environment variables, not hardcoded!)
# Set these in your local machine: 
#   $env:OPENAI_API_KEY="your_key_here"   (PowerShell)
#   export OPENAI_API_KEY="your_key_here" (Linux/Mac)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "")

# Gmail credentials also from env (never hardcode in repo!)
GMAIL_ADDRESS = os.getenv("GMAIL_ADDRESS", "")
GMAIL_APP_PASSWORD = os.getenv("GMAIL_APP_PASSWORD", "")

# 📧 Pydantic model for clarity
class Email(BaseModel):
    subject: str
    body: str
    to: str

# 📧 Gmail tool (HTML enabled)
@function_tool
def sendEmail(subject: str, body: str, to: str):
    msg = EmailMessage()
    msg["From"] = GMAIL_ADDRESS
    msg["To"] = to
    msg["Subject"] = subject

    msg.set_content("This email requires an HTML-compatible client.")
    msg.add_alternative(body, subtype="html")

    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(GMAIL_ADDRESS, GMAIL_APP_PASSWORD)
        server.send_message(msg)

    return "✅ Email sent successfully!"

# 🧠 Agents
information_agent = Agent(
    name="information_agent",
    instructions="You are a helpful assistant that gives detailed factual answers."
)

email_agent = Agent(
    name="email_agent",
    instructions="""
    You are an AI assistant that writes professional HTML emails.
    Always produce valid HTML with inline CSS.
    """,
    tools=[sendEmail],
    output_type=Email
)

runner = Runner()

# ⚡ In Jupyter, use asyncio.run instead of `await` directly
with trace("tracing"):
    info_result = asyncio.run(runner.run(
        information_agent,
        "What is the capital of France?"
    ))

    email_result = asyncio.run(runner.run(
        email_agent,
        f"Subject: Capital of France\nBody: The capital of France is {info_result.final_output}.\nSend this to something@gmail.com"
    ))

print("📨 Final Output:", email_result.final_output)


📨 Final Output: subject='Capital of France' body='<html>\n  <body style="font-family:Arial, sans-serif; line-height:1.6; color:#333; padding:20px;">\n    <div style="max-width:600px; margin:0 auto; border:1px solid #e0e0e0; border-radius:8px; overflow:hidden;">\n      <div style="background:#4CAF50; color:white; padding:15px; text-align:center; font-size:20px;">\n        📩 AI Information Service\n      </div>\n      <div style="padding:20px;">\n        <h2 style="color:#4CAF50; margin-top:0;">Capital of France</h2>\n        <p>The capital of France is The capital of France is Paris..</p>\n      </div>\n      <div style="background:#f4f4f4; text-align:center; padding:10px; font-size:12px; color:#777;">\n        This email was generated by AI • Do not reply\n      </div>\n    </div>\n  </body>\n</html>' to='foziajavaid3436@gmail.com'


In [40]:
import agents.tool
print(agents.tool)
print(dir(agents.tool))


<module 'agents.tool' from 'c:\\D\\mail agent\\.venv\\Lib\\site-packages\\agents\\tool.py'>
['Any', 'AsyncComputer', 'Awaitable', 'Callable', 'CodeInterpreter', 'CodeInterpreterTool', 'Computer', 'ComputerTool', 'ComputerToolSafetyCheckData', 'Concatenate', 'DocstringStyle', 'FileSearchTool', 'Filters', 'FunctionTool', 'FunctionToolResult', 'HostedMCPTool', 'ImageGeneration', 'ImageGenerationTool', 'Literal', 'LocalShellCall', 'LocalShellCommandRequest', 'LocalShellExecutor', 'LocalShellTool', 'MCPToolApprovalFunction', 'MCPToolApprovalFunctionResult', 'MCPToolApprovalRequest', 'MaybeAwaitable', 'Mcp', 'McpApprovalRequest', 'ModelBehaviorError', 'NotRequired', 'ParamSpec', 'PendingSafetyCheck', 'RankingOptions', 'ResponseComputerToolCall', 'RunContextWrapper', 'RunItem', 'SpanError', 'TYPE_CHECKING', 'Tool', 'ToolContext', 'ToolErrorFunction', 'ToolFunction', 'ToolFunctionWithContext', 'ToolFunctionWithToolContext', 'ToolFunctionWithoutContext', 'ToolParams', 'TypedDict', 'Union', 'Use

subject='Key Highlights of Paris, the Capital of France' body='Dear [Recipient],\n\nI wanted to share a brief overview of Paris, the capital of France. Beyond serving as France’s political and administrative center, Paris is renowned as a cultural, economic, and historical hub. Often referred to as "The City of Light," it stands out as one of the most influential cities globally, shaping art, fashion, and ideas.\n\nPlease let me know if you need further information or specific details about Paris.\n\nBest regards,\n[Your Name]'


Agent and Runner ready
